In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, CovarianceStructure, ObjectiveLib
import matplotlib.pyplot as plt
import pickle 

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

Names in str: 'Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)'

In [2]:
Nt =110 # number of timepoints

max_manual_num = 5  # max manual number per dynamical measurement
min_interval_num = 10 # minimal time interval between dynamical timepoints

In [3]:
static_ind = [0,1,2,3,4,5,6,7,8,9,10] # column number for static measurements 
dynamic_ind = [11,12,13,14,15] # column number for dynamic measurements 
all_ind = static_ind+dynamic_ind

num_total = len(all_ind)

In [4]:
# string names for measurements
all_names_strategy3 = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T',  'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', # all static
            'Ads.gas_outlet.z("CO2").static', 'Des.gas_outlet.z("CO2").static', # static z 
            'Ads.gas_outlet.z("CO2").dynamic', 'Des.gas_outlet.z("CO2").dynamic', # dynamic z 
            'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [5]:

static_cost = [1000, #ads.gas_inlet.F (0)
    1000, #ads.gas_outlet.F (1)
     500, #ads.gas_outlet.T (2)
    1000, #des.gas_inlet.F (4)
    1000, #des.gas_outlet.F (5)
     500, #des.gas_outlet.T (6)
     1000, #ads.T19 (8)
     1000, #ads.T23 (9)
     1000, #ads.T28 (10)
    7000,
    7000]

# static cost are all installatin costs 
static_cost.extend([100, 100, 500, 500, 500])

# dynamic cost are for timepoints
dynamic_cost = [0]*len(static_ind)
dynamic_cost.extend([100]*len(dynamic_ind))

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
measure_info = pd.DataFrame({
    "name": all_names_strategy3,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                               name  Q_index  static_cost  dynamic_cost  \
0                   Ads.gas_inlet.F        0         1000             0   
1                  Ads.gas_outlet.F        1         1000             0   
2                  Ads.gas_outlet.T        2          500             0   
3                   Des.gas_inlet.F        3         1000             0   
4                  Des.gas_outlet.F        4         1000             0   
5                  Des.gas_outlet.T        5          500             0   
6              Ads.T_g.Value(19,10)        6         1000             0   
7              Ads.T_g.Value(23,10)        7         1000             0   
8              Ads.T_g.Value(28,10)        8         1000             0   
9    Ads.gas_outlet.z("CO2").static        9         7000             0   
10   Des.gas_outlet.z("CO2").static       10         7000             0   
11  Ads.gas_outlet.z("CO2").dynamic       11          100           100   
12  Des.gas_outlet.z("CO2

In [7]:
dataObject = DataProcess()
dataObject.read_jacobian('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.get_Q_list([0,1,2,4,5,6,8,9,10,3,7],
                        [3,7,11,12,13], Nt)


In [8]:
calculator = MeasurementOptimizer(Q, measure_info, error_opt=CovarianceStructure.identity, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


### Initialize

In [9]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [10]:
# initialize cov_y
with open('MarResult-strategy3-Dopt/Mar31_D_110_27700', 'rb') as f:
    x = pickle.load(f)
    
print(x)
init_cov_y = x.copy()

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
# initialize FIM:
with open('MarResult-strategy3-Dopt/Mar31_FIM_D_110_27700', 'rb') as f:
    fim_prior = pickle.load(f)
    
print(fim_prior)

[[ 5.55260011e+03 -2.61972445e+00 -4.14130933e+03  1.09879444e+04
  -5.00376769e+02]
 [-2.61972445e+00  5.22032483e+00  7.68731822e+00  5.03877570e+01
  -1.05219552e+01]
 [-4.14130933e+03  7.68731822e+00  3.32562801e+03 -8.52086707e+03
   1.66399892e+02]
 [ 1.09879444e+04  5.03877570e+01 -8.52086707e+03  2.52492460e+04
  -8.96253626e+02]
 [-5.00376769e+02 -1.05219552e+01  1.66399892e+02 -8.96253626e+02
   3.78315460e+02]]


In [12]:

mip_option = True
objective = ObjectiveLib.A
sparse_opt = True
fix_opt = False

manual_num = 20
budget_opt = 1000

num_dynamic_time = np.linspace(2,220,Nt)

static_dynamic = [[9,11], [10,12]]
time_interval_for_all = True

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 

In [13]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    upper_diagonal_only=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    init_cov_y= init_cov_y, 
                    initial_fim = fim_prior,
                    static_dynamic_pair=static_dynamic,
                    time_interval_all_dynamic = time_interval_for_all)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

In [14]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.55260011e+03 -2.61972445e+00 -4.14130933e+03  1.09879444e+04
  -5.00376769e+02]
 [-2.61972445e+00  5.22032483e+00  7.68731822e+00  5.03877570e+01
  -1.05219552e+01]
 [-4.14130933e+03  7.68731822e+00  3.32562801e+03 -8.52086707e+03
   1.66399892e+02]
 [ 1.09879444e+04  5.03877570e+01 -8.52086707e+03  2.52492460e+04
  -8.96253626e+02]
 [-5.00376769e+02 -1.05219552e+01  1.66399892e+02 -8.96253626e+02
   3.78315460e+02]]
trace: 34511.00993581643
det: 2119056400016.21
[3.31938994e+04 8.18378909e+02 4.48378213e+02 4.66218092e+01
 3.73160950e+00]
Pyomo OF: 34511.00993581643
Log_det: 28.38199201119661


In [15]:
ans_y, sol_y = calculator.extract_solutions(mod)


Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  1.0
Ads.gas_outlet.T :  1.0
Des.gas_inlet.F :  1.0
Des.gas_outlet.F :  1.0
Des.gas_outlet.T :  1.0
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  1.0
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2").static :  1.0
Des.gas_outlet.z("CO2").static :  1.0
Ads.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Des.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0.

In [16]:
store = True

if store:
    file = open('Apr17_A_mip_1000', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('Apr17_FIM_A_mip_1000', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()